In [ ]:
# neet-network-example.ipynb
#
# Bryan Daniels
# 2022/6/29 - 2023/5/15
#
# Examples of setting up networks from scratch in neet and computing control kernels.
#

In [1]:
from neet import boolean

# Creating a logic network

A `LogicNetwork` works using a logic "table".  Each row of the logic table specifies one node's inputs and which combinations of those inputs will result in that node becoming active (see the documentation for `boolean.LogicNetwork` for more details).  The Cell Collective networks are implemented as logic networks.

Here's a random example of a logic table for a network of size 3:

In [2]:
table = [((1,2),{"00","01"}),
         ((0,1),{"11"}),
         ((1,), {"0"})]

In [3]:
net1 = boolean.LogicNetwork(table)

In [4]:
net1.update([0,0,1])

[1, 0, 1]

# Creating a weight-threshold network

Another type of network is the "weight-threshold" network (`boolean.WTnetwork`).  In this type, you specify a weight matrix for how strongly the state of node `i` influences node `j`, along with a threshold for each node.  A node becomes active when the sum of incoming signals from other nodes exceeds its threshold (see the documentation for `boolean.WTNetwork` for more details).

A random example of a weight-threshold network (with different dynamics than the above network):

In [5]:
weights = [[0,2,1],
           [1,5,-2],
           [0,1,0]]
thresholds = [1,-1,0]

In [6]:
net2 = boolean.WTNetwork(weights,thresholds)

In [7]:
net2.update([0,0,1])

[0, 0, 1]

# Creating a network from a transition list

Alternatively, one may define a network's dynamics by explicitly specifying the transition map for all network states.  Here, we start with a list of length $2^{(\mathrm{\#~of~nodes)}}$, where the entry at index $i$ gives the resulting network state at time $t+1$ given that the network starts at state $i$ at time $t$.  (Network states are encoded as integers using `net.encode`, and decoded back to binary lists using `net.decode`.)

Note that the transition map does not uniquely define the network connectivity.  That is, there are in general many networks that produce the same dynamics.  The technique we use here for defining the dynamics simply results in each node's future state depending on the state of all nodes in the network.

Here's an example network with 3 nodes and 4 fixed point attractors:

In [8]:
from neet.boolean.TransitionNetwork import transitions_to_net

In [9]:
transitions = [0,1,2,3,0,1,2,3]
net3 = transitions_to_net(transitions)
net3.attractors

array([[0],
       [1],
       [2],
       [3]])

# Computing control kernels

In [10]:
from neet.controlkernel.control_kernel_analysis import ck_analysis

In [11]:
ck_analysis(net3)['control_kernels']

Finding attractors and control kernels...


[{0, 1}, {0, 1}, {0, 1}, {0, 1}]

In [12]:
for i,net in enumerate([net1,net2,net3]):
    print("---------")
    print("Network {}".format(i+1))
    print("---------")
    ck_data = ck_analysis(net)
    print("Attractors: {}".format(ck_data['attractors']))
    print("Control kernels: {}".format(ck_data['control_kernels']))
    print()

---------
Network 1
---------
Finding attractors and control kernels...
Attractors: [[5]]
Control kernels: [set()]

---------
Network 2
---------
Finding attractors and control kernels...
Attractors: [[7]
 [4]
 [5]]
Control kernels: [{1}, {0, 1, 2}, {0, 1, 2}]

---------
Network 3
---------
Finding attractors and control kernels...
Attractors: [[0]
 [1]
 [2]
 [3]]
Control kernels: [{0, 1}, {0, 1}, {0, 1}, {0, 1}]

